In [ ]:
!pip install transformers torch
from transformers import pipeline

# Load multilingual sentiment analysis model
sentiment_pipeline = pipeline("sentiment-analysis", model="cardiffnlp/twitter-xlm-roberta-base-sentiment")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 37.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

Device set to use cpu


In [ ]:
import pandas as pd

# Load dataset
file_path = "/content/translated_comments_dataset.csv"  # Change if needed
df = pd.read_csv(file_path)

# Ensure text data is string
df["translated_comments"] = df["translated_comments"].astype(str)

# Function for sentiment analysis
def predict_multilingual_sentiment(text):
    if not text.strip():
        return "Neutral"

    # Limit text length to avoid tensor mismatch errors
    max_length = 512  # Safe limit for transformer models
    text = text[:max_length]

    result = sentiment_pipeline(text)[0]
    return result["label"]

# Apply sentiment analysis with fixed function
df["multilingual_sentiment"] = df["translated_comments"].apply(predict_multilingual_sentiment)


In [ ]:
sentiment_distribution = df["multilingual_sentiment"].value_counts(normalize=True) * 100
print(sentiment_distribution)


multilingual_sentiment
positive    47.196871
neutral     35.984355
negative    16.818774
Name: proportion, dtype: float64


In [ ]:
print(df.columns)


Index(['video_id', 'title', 'description', 'published_at', 'channel_id',
       'channel_title', 'category_id', 'tags', 'duration', 'definition',
       'caption', 'view_count', 'like_count', 'dislike_count',
       'favorite_count', 'comment_count', 'category_name', 'comments',
       'fetch_date', 'translated_comments', 'multilingual_sentiment'],
      dtype='object')


In [ ]:
category_sentiment_distribution = category_sentiment_distribution.fillna(0)


In [ ]:
# Calculate sentiment distribution for each category
category_sentiment_distribution = df.groupby("category_name")["multilingual_sentiment"].value_counts(normalize=True).unstack() * 100

# Display the category-wise sentiment analysis
category_sentiment_distribution


multilingual_sentiment,negative,neutral,positive
category_name,,,
Autos & Vehicles,NaN,100.000000,NaN
Comedy,19.444444,33.333333,47.222222
Education,7.142857,78.571429,14.285714
Entertainment,21.574344,32.944606,45.481050
Film & Animation,12.500000,50.000000,37.500000
Gaming,7.500000,60.000000,32.500000
Howto & Style,9.302326,30.232558,60.465116
Music,3.658537,24.390244,71.951220
News & Politics,48.148148,25.925926,25.925926


In [ ]:
missing_categories = df["category_name"].isna().sum()
print(f"Missing category labels: {missing_categories}")


Missing category labels: 0


In [ ]:
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import numpy as np

# Load the sentiment model and tokenizer
model_name = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Function for sentiment analysis
def analyze_sentiment(text):
    # Ensure text is a string and handle NaN values
    if isinstance(text, str):
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
        scores = outputs.logits.softmax(dim=-1).numpy()[0]
        labels = ["negative", "neutral", "positive"]
        return labels[np.argmax(scores)]
    else:  # Handle NaN or other non-string values
        return "neutral"  # You might want to choose a different default sentiment here

# Load the cleaned dataset (Ensure the file is in the same directory)
file_path = "/content/cleaned_translated_comments.csv"
df = pd.read_csv(file_path)

# Run sentiment analysis on each comment, converting to string before applying
df["sentiment"] = df["cleaned_comments"].astype(str).apply(analyze_sentiment)

# Print some sample results
print("\nSAMPLE SENTIMENT RESULTS:")
print(df[["cleaned_comments", "sentiment"]].head(10))

# Load the original dataset to get category-wise analysis
original_file_path = "/content/translated_comments_dataset.csv"  # Update to the original dataset
df_original = pd.read_csv(original_file_path)

# Merge sentiment results with original dataset using 'translated_comments' as the key
df_merged = df_original.merge(df, left_on="translated_comments", right_on="cleaned_comments", how="inner")

# Category-wise sentiment distribution
category_sentiment = df_merged.groupby("category_name")["sentiment"].value_counts().unstack()

print("\nCATEGORY-WISE SENTIMENT ANALYSIS:")
print(category_sentiment)

# Save the results
output_file = "sentiment_analysis_results2.csv"
df_merged.to_csv(output_file, index=False)
print(f"\nSentiment analysis completed. Results saved to {output_file}")


SAMPLE SENTIMENT RESULTS:
                                    cleaned_comments sentiment
0  In the past, politics is less and more sociall...  negative
1  India US & UK Tour 2025 Tickets Link: https://...   neutral
2                                       Awesome orki  positive
3  Such me bro, kya njrana pesh kiya he, mja aagy...  positive
4  Akash bhai was not expected from you. ... that...  negative
5                             Kyu Nhi Hoi stand up 😢  negative
6                   Bcci ko selector badalna chahiye  negative
7               Rohit se batting me bumrah achha hai  positive
8  Rohit besharm ko bhagao virat aur pant vi nika...  negative
9  these reepentitative collapses Drop Non Perfor...  negative


FileNotFoundError: [Errno 2] No such file or directory: '/content/translated_comments_dataset.csv'